In [1]:
import torch
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

True


device(type='cuda')

In [14]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234
torch.manual_seed(SEED) #为CPU设置随机种子
torch.cuda.manual_seed(SEED)#为GPU设置随机种子
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy',tokenizer_language='en_core_web_sm')
LABEL = data.LabelField(dtype=torch.float)

In [15]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [16]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(SEED)) #默认split_ratio=0.7

In [17]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [18]:
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data) 

100%|█████████▉| 399999/400000 [00:20<00:00, 19617.64it/s]


In [19]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [22]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
                                                        (train_data, valid_data, test_data), 
                                                        batch_size=BATCH_SIZE,
                                                        device=device)

In [25]:
import torch.nn as nn
import torch.nn.functional as F

class WordAVGModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(embedding_dim, output_dim) #output_dim=1
        
    def forward(self, text): # text: [seq_len,batch_size]
        embedded = self.embedding(text)  # embedded = [seq_len, batch_size, embedding_dim] 
        embedded = embedded.permute(1, 0, 2) # [batch_size, seq_len, embedding_dim]  
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)   
        return self.fc(pooled)  # batch_size, output_dim

In [26]:
VOCAB_SZIE = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1 
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] 

model = WordAVGModel(VOCAB_SZIE, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [27]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [28]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3896, -0.0554,  0.4922,  ..., -0.0182, -0.8245,  0.0696],
        [ 0.2828,  0.4366,  0.7027,  ..., -1.0296, -0.0782,  0.6539],
        [ 2.4727, -0.4058,  0.2144,  ...,  0.5650,  1.4462, -0.9035]])

In [29]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token] # UNK_IDX=0

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM) 
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [30]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters()) 
loss_fn = nn.BCEWithLogitsLoss()  #BCEWithLogitsLoss：二分类损失函数
model = model.to(device) 
loss_fn = loss_fn.to(device) 

In [31]:
def binary_accuracy(preds, y): 
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum()/len(correct)
    return acc


def train(model, iterator, optimizer, loss_fn):
    epoch_loss = 0
    epoch_acc = 0
    total_len = 0
    model.train() 
    for batch in iterator: 
        optimizer.zero_grad() 
        preds = model(batch.text).squeeze(1)
        loss = loss_fn(preds, batch.label)
        acc = binary_accuracy(preds, batch.label)
        loss.backward() 
        optimizer.step() 
        epoch_loss += loss.item() * len(batch.label)
        epoch_acc += acc.item() * len(batch.label)
        total_len += len(batch.label)   
    return epoch_loss / total_len, epoch_acc / total_len

In [32]:
def evaluate(model, iterator, loss_fn):
    epoch_loss = 0
    epoch_acc = 0
    total_len = 0
    model.eval()
    with torch.no_grad():  
        for batch in iterator: 
            
            preds = model(batch.text).squeeze(1)
            loss = loss_fn(preds, batch.label)
            acc = binary_accuracy(preds, batch.label)
            
            epoch_loss += loss.item() * len(batch.label)
            epoch_acc += acc.item() * len(batch.label)
            total_len += len(batch.label)
    model.train()
    return epoch_loss / total_len, epoch_acc / total_len

In [33]:
import time 

def epoch_time(start_time, end_time): 
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
from tqdm import tqdm
N_EPOCHS = 20
best_valid_loss = float('inf') 

for epoch in tqdm(range(N_EPOCHS)):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn) # 得到训练集每个epoch的平均损失和准确率
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn) # 验证集每个epoch的平均损失和准确率，model传入的是训练完的参数
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss: 
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'wordavg-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  5%|▌         | 1/20 [00:05<01:49,  5.76s/it]

Epoch: 01 | Epoch Time: 0m 5s
	Train Loss: 0.686 | Train Acc: 59.51%
	 Val. Loss: 0.625 |  Val. Acc: 69.67%


 10%|█         | 2/20 [00:11<01:44,  5.80s/it]

Epoch: 02 | Epoch Time: 0m 5s
	Train Loss: 0.642 | Train Acc: 75.42%
	 Val. Loss: 0.511 |  Val. Acc: 76.40%


 15%|█▌        | 3/20 [00:17<01:38,  5.77s/it]

...

Epoch: 18 | Epoch Time: 0m 4s
	Train Loss: 0.159 | Train Acc: 95.15%
	 Val. Loss: 0.702 |  Val. Acc: 88.80%


 95%|█████████▌| 19/20 [01:39<00:05,  5.03s/it]

Epoch: 19 | Epoch Time: 0m 4s
	Train Loss: 0.151 | Train Acc: 95.45%
	 Val. Loss: 0.723 |  Val. Acc: 88.73%


100%|██████████| 20/20 [01:44<00:00,  5.22s/it]

Epoch: 20 | Epoch Time: 0m 4s
	Train Loss: 0.142 | Train Acc: 95.83%
	 Val. Loss: 0.741 |  Val. Acc: 89.05%


In [36]:
model.load_state_dict(torch.load("wordavg-model.pt"))

<All keys matched successfully>

In [37]:
model.load_state_dict(torch.load('wordavg-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.431 | Test Acc: 82.54%


In [38]:
import spacy  
nlp = spacy.load('en_core_web_sm') 

def predict_sentiment(sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    indexed = [TEXT.vocab.stoi[t] for t in tokenized] 
    tensor = torch.LongTensor(indexed).to(device) #seq_len
    tensor = tensor.unsqueeze(1)    
    pred = torch.sigmoid(model(tensor))
    return pred.item()

In [39]:
predict_sentiment("This film is terrible")

0.0001455277670174837

In [40]:
predict_sentiment("This film is great")

1.0

In [41]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text): # text=[seq_len, batch_size]
        embedded = self.dropout(self.embedding(text)) #[seq_len, batch_size, embedding_dim]
        output, (hidden, cell) = self.lstm(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))    
        return self.fc(hidden.squeeze(0)) # #sequeeze()只有维度为1才会去掉，输出[batch size, output_dim]

In [42]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [43]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [44]:
# 同上初始化
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3896, -0.0554,  0.4922,  ..., -0.0182, -0.8245,  0.0696],
        [ 0.2828,  0.4366,  0.7027,  ..., -1.0296, -0.0782,  0.6539],
        [ 2.4727, -0.4058,  0.2144,  ...,  0.5650,  1.4462, -0.9035]])


In [45]:
optimizer = optim.Adam(model.parameters())
model = model.to(device)

In [46]:
# 用kaggleGPU跑花了40分钟。
N_EPOCHS = 20
best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS)):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  5%|▌         | 1/20 [00:46<14:43, 46.48s/it]

Epoch: 01 | Epoch Time: 0m 46s
	Train Loss: 0.679 | Train Acc: 56.73%
	 Val. Loss: 0.633 |  Val. Acc: 63.69%


 10%|█         | 2/20 [01:32<13:50, 46.15s/it]

Epoch: 02 | Epoch Time: 0m 45s
	Train Loss: 0.673 | Train Acc: 56.59%
	 Val. Loss: 0.678 |  Val. Acc: 61.99%


 15%|█▌        | 3/20 [02:19<13:08, 46.37s/it]

...

Epoch: 18 | Epoch Time: 0m 47s
	Train Loss: 0.063 | Train Acc: 97.77%
	 Val. Loss: 0.389 |  Val. Acc: 89.41%


 95%|█████████▌| 19/20 [14:53<00:47, 47.00s/it]

Epoch: 19 | Epoch Time: 0m 46s
	Train Loss: 0.058 | Train Acc: 98.14%
	 Val. Loss: 0.405 |  Val. Acc: 89.24%


100%|██████████| 20/20 [15:41<00:00, 47.07s/it]

Epoch: 20 | Epoch Time: 0m 47s
	Train Loss: 0.052 | Train Acc: 98.22%
	 Val. Loss: 0.412 |  Val. Acc: 89.00%


In [47]:
model.load_state_dict(torch.load('lstm-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.305 | Test Acc: 87.90%


In [48]:
predict_sentiment("I feel This film bad")

0.02666657418012619

In [49]:
predict_sentiment("This film is great")

0.9967237114906311

In [50]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels = 1, out_channels = n_filters, 
                                    kernel_size = (fs, embedding_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        text = text.permute(1, 0) # [batch size, sent len]
        embedded = self.embedding(text) # [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1) # [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]      
        cat = self.dropout(torch.cat(pooled, dim=1))
  
        return self.fc(cat)

In [51]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
model = model.to(device)
print(f'The model has {count_parameters(model):,} trainable parameters')
# 比averge model模型参数差不多

The model has 2,620,801 trainable parameters


In [52]:
# 同上，需要花8分钟左右
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'CNN-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/root/miniconda3/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.649 | Train Acc: 61.61%
	 Val. Loss: 0.507 |  Val. Acc: 78.01%
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.423 | Train Acc: 80.55%
	 Val. Loss: 0.367 |  Val. Acc: 83.43%
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 0.304 | Train Acc: 87.35%
	 Val. Loss: 0.330 |  Val. Acc: 85.68%
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.220 | Train Acc: 91.26%
	 Val. Loss: 0.323 |  Val. Acc: 86.89%
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.162 | Train Acc: 93.90%
	 Val. Loss: 0.335 |  Val. Acc: 86.88%
Epoch: 06 | Epoch Time: 0m 12s
	Train Loss: 0.109 | Train Acc: 96.12%
	 Val. Loss: 0.352 |  Val. Acc: 86.89%
Epoch: 07 | Epoch Time: 0m 12s
	Train Loss: 0.080 | Train Acc: 97.23%
	 Val. Loss: 0.371 |  Val. Acc: 87.05%
Epoch: 08 | Epoch Time: 0m 12s
	Train Loss: 0.056 | Train Acc: 98.25%
	 Val. Loss: 0.400 |  Val. Acc: 87.48%
Epoch: 09 | Epoch Time: 0m 12s
	Train Loss: 0.044 | Train Acc: 98.71%
	 Val. Loss: 0.425 |  Val. Acc: 86.93%
Epoch: 10 | Epoch T

In [53]:
model.load_state_dict(torch.load('CNN-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.340 | Test Acc: 85.48%


In [54]:
predict_sentiment("I feel This film bad")

0.36543038487434387

In [55]:
predict_sentiment("This film is great well") 

0.9817049503326416